# Comprehensive Run Analysis Notebook

This notebook provides a complete analysis of any Olmo Conformity experiment run.

**Features:**
- Run metadata and configuration
- Behavioral trial analysis with examples
- Activation analysis from safetensors files
- Probe projection analysis
- Logit lens analysis
- Think token analysis (for Think variants)
- Intervention results
- Sample prompts and responses

**Usage:**
1. Set `RUN_ID` in the cell below
2. Run all cells to get comprehensive analysis

## Configuration: Set Your Run ID

**Run this cell first.** Set `RUN_ID` to match your run. You can use:
- Full directory name: `20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631`
- Just the UUID: `0af03fbc-d576-4afa-9815-b37a11f57631`
- Or set `RUN_DIR` to the full path to the run folder

Then run all remaining cells for full analysis (DB, Safetensors, prompts, responses, probes, interventions).

In [ ]:
# ============================================================================
# CONFIGURATION: Set your run ID here
# ============================================================================

# Option 1: Provide full run ID (timestamp_UUID format)
RUN_ID = "20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631"

# Option 2: Or provide just the UUID part (will search for matching run directory)
# RUN_ID = "0af03fbc-d576-4afa-9815-b37a11f57631"

# Option 3: Or provide run directory path directly
# RUN_DIR = "/path/to/runs/20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631"

# Base runs directory (default: ./runs)
RUNS_DIR = "./runs"

print(f"Run ID: {RUN_ID}")
print(f"Runs directory: {RUNS_DIR}")

Run ID: 20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631
Runs directory: ./runs


## Setup: Imports and Utilities

In [2]:
# ============================================================================
# SETUP: Imports and utilities
# ============================================================================

import json
import os
import sys
from pathlib import Path
from typing import Dict, List, Optional, Any
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Add repo root and src to path for imports
REPO_ROOT = Path.cwd().parent if (Path.cwd() / "utils.py").exists() or Path.cwd().name == "Analysis Scripts" else Path.cwd()
if (REPO_ROOT / "src" / "aam").exists():
    pass  # REPO_ROOT is correct
else:
    REPO_ROOT = Path.cwd()
SRC_ROOT = REPO_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

# Import AAM modules
from aam.persistence import TraceDb, TraceDbConfig
from aam.analytics.utils import load_simulation_db, check_missing_prerequisites

# Import utils for run resolution (from Analysis Scripts folder)
sys.path.insert(0, str(REPO_ROOT / "Analysis Scripts"))
from utils import find_run_dir_for_run_id, resolve_run_ref

# Safetensors for activation loading
try:
    from safetensors.torch import load_file as load_safetensors
    import torch
    SAFETENSORS_AVAILABLE = True
except ImportError:
    SAFETENSORS_AVAILABLE = False
    print("⚠️  safetensors not installed - activation analysis will be limited")

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Resolve RUNS_DIR relative to repo root if it looks relative
if not os.path.isabs(RUNS_DIR):
    RUNS_DIR = str(REPO_ROOT / "runs")

print("✅ Imports successful")
print(f"   REPO_ROOT: {REPO_ROOT}")
print(f"   RUNS_DIR: {RUNS_DIR}")

✅ Imports successful
   REPO_ROOT: /Users/mahdi/repos/abstractAgentMachine
   RUNS_DIR: /Users/mahdi/repos/abstractAgentMachine/runs


## Step 1: Find and Load Run Directory

In [3]:
# ============================================================================
# STEP 1: Find and load run directory
# ============================================================================

# Resolve run directory
try:
    if 'RUN_DIR' in globals():
        run_ref = resolve_run_ref(run_id=None, run_dir=RUN_DIR, runs_dir=RUNS_DIR)
    else:
        run_ref = resolve_run_ref(run_id=RUN_ID, run_dir=None, runs_dir=RUNS_DIR)
    
    RUN_DIR = run_ref.run_dir
    RUN_ID = run_ref.run_id
    
    print(f"✅ Found run directory: {RUN_DIR}")
    print(f"✅ Run ID: {RUN_ID}")
except Exception as e:
    print(f"❌ Error finding run: {e}")
    raise

# Verify paths
DB_PATH = Path(RUN_DIR) / "simulation.db"
ACTIVATIONS_DIR = Path(RUN_DIR) / "activations"
ARTIFACTS_DIR = Path(RUN_DIR) / "artifacts"

print(f"\n📁 Paths:")
print(f"   Database: {DB_PATH} (exists: {DB_PATH.exists()})")
print(f"   Activations: {ACTIVATIONS_DIR} (exists: {ACTIVATIONS_DIR.exists()})")
print(f"   Artifacts: {ARTIFACTS_DIR} (exists: {ARTIFACTS_DIR.exists()})")

✅ Found run directory: /Users/mahdi/repos/abstractAgentMachine/runs/20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631
✅ Run ID: 20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631

📁 Paths:
   Database: /Users/mahdi/repos/abstractAgentMachine/runs/20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631/simulation.db (exists: True)
   Activations: /Users/mahdi/repos/abstractAgentMachine/runs/20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631/activations (exists: True)
   Artifacts: /Users/mahdi/repos/abstractAgentMachine/runs/20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631/artifacts (exists: True)


## Step 2: Load Database and Check Prerequisites

In [4]:
# ============================================================================
# STEP 2: Load database and check prerequisites
# ============================================================================

# Load database
trace_db = load_simulation_db(RUN_DIR)
print("✅ Database loaded")

# Check what data is available
prerequisites = check_missing_prerequisites(trace_db, RUN_ID)
print("\n📊 Available data:")
for name, exists in prerequisites.items():
    status = "✅" if exists else "❌"
    print(f"   {status} {name}: {exists}")

# Get run metadata
run_meta = trace_db.conn.execute(
    "SELECT run_id, seed, created_at, config_json FROM runs WHERE run_id = ?",
    (RUN_ID,)
).fetchone()

if run_meta:
    config = json.loads(run_meta["config_json"])
    print(f"\n📋 Run Metadata:")
    print(f"   Run ID: {run_meta['run_id']}")
    print(f"   Seed: {run_meta['seed']}")
    print(f"   Created: {datetime.fromtimestamp(run_meta['created_at'])}")
    print(f"   Config mode: {config.get('mode', 'unknown')}")
    if 'suite_config' in config:
        suite_cfg = config['suite_config']
        print(f"   Temperature: {suite_cfg.get('run', {}).get('temperature', 'unknown')}")
        print(f"   Models: {len(suite_cfg.get('models', []))}")
        print(f"   Conditions: {len(suite_cfg.get('conditions', []))}")
else:
    print("❌ Run not found in database")
    raise ValueError(f"Run {RUN_ID} not found")

✅ Database loaded

📊 Available data:
   ❌ judgeval_scores: False
   ❌ probes: False
   ❌ probe_projections: False
   ❌ interventions: False
   ❌ intervention_results: False
   ❌ think_tokens: False
   ❌ logit_lens: False
   ❌ activation_capture: False
❌ Run not found in database


ValueError: Run 20260124_230102_0af03fbc-d576-4afa-9815-b37a11f57631 not found

## Section 1: Run Overview and Statistics

In [ ]:
# ============================================================================
# Run Statistics Overview
# ============================================================================

# Count trials by variant and condition
trials_summary = pd.read_sql_query("""
    SELECT 
        t.variant,
        c.name as condition_name,
        COUNT(*) as trial_count,
        SUM(CASE WHEN o.is_correct = 1 THEN 1 ELSE 0 END) as correct_count,
        SUM(CASE WHEN o.is_correct = 0 THEN 1 ELSE 0 END) as incorrect_count,
        SUM(CASE WHEN o.is_correct IS NULL THEN 1 ELSE 0 END) as null_count,
        AVG(CASE WHEN o.is_correct = 1 THEN 1.0 ELSE 0.0 END) as accuracy
    FROM conformity_trials t
    LEFT JOIN conformity_conditions c ON c.condition_id = t.condition_id
    LEFT JOIN conformity_outputs o ON o.trial_id = t.trial_id
    WHERE t.run_id = ?
    GROUP BY t.variant, c.name
    ORDER BY t.variant, c.name
""", trace_db.conn, params=(RUN_ID,))

print("📊 Trial Summary by Variant and Condition:")
print(trials_summary.to_string(index=False))

# Overall statistics
total_trials = pd.read_sql_query("""
    SELECT COUNT(*) as total FROM conformity_trials WHERE run_id = ?
""", trace_db.conn, params=(RUN_ID,)).iloc[0]['total']

total_outputs = pd.read_sql_query("""
    SELECT COUNT(*) as total 
    FROM conformity_outputs o
    JOIN conformity_trials t ON t.trial_id = o.trial_id
    WHERE t.run_id = ?
""", trace_db.conn, params=(RUN_ID,)).iloc[0]['total']

print(f"\n📈 Overall Statistics:")
print(f"   Total trials: {total_trials}")
print(f"   Total outputs: {total_outputs}")
print(f"   Variants tested: {trials_summary['variant'].nunique()}")
print(f"   Conditions tested: {trials_summary['condition_name'].nunique()}")

## Section 2: Sample Prompts and Responses

In [ ]:
# ============================================================================
# Extract Sample Prompts and Responses
# ============================================================================

# Get sample trials with full prompt and response data
samples = pd.read_sql_query("""
    SELECT 
        t.trial_id,
        t.variant,
        t.model_id,
        t.temperature,
        c.name as condition_name,
        i.question,
        i.ground_truth_text,
        i.domain,
        p.system_prompt,
        p.user_prompt,
        p.chat_history_json,
        o.raw_text as response,
        o.parsed_answer_text,
        o.is_correct,
        o.refusal_flag,
        o.latency_ms,
        o.parsed_answer_json
    FROM conformity_trials t
    JOIN conformity_conditions c ON c.condition_id = t.condition_id
    JOIN conformity_items i ON i.item_id = t.item_id
    LEFT JOIN conformity_prompts p ON p.trial_id = t.trial_id
    LEFT JOIN conformity_outputs o ON o.trial_id = t.trial_id
    WHERE t.run_id = ?
    ORDER BY t.created_at
    LIMIT 50
""", trace_db.conn, params=(RUN_ID,))

print(f"📝 Loaded {len(samples)} sample trials")

# Display samples by condition
for condition in samples['condition_name'].unique():
    if pd.isna(condition):
        continue
    cond_samples = samples[samples['condition_name'] == condition].head(3)
    print(f"\n{'='*80}")
    print(f"📋 Condition: {condition}")
    print(f"{'='*80}")
    
    for idx, row in cond_samples.iterrows():
        print(f"\n--- Sample {idx + 1} ---")
        print(f"Variant: {row['variant']} | Domain: {row['domain']} | Temp: {row['temperature']}")
        print(f"\nQuestion: {row['question']}")
        print(f"Ground Truth: {row['ground_truth_text']}")
        print(f"\nSystem Prompt:")
        print(f"  {row['system_prompt']}")
        print(f"\nUser Prompt:")
        print(f"  {row['user_prompt']}")
        print(f"\nResponse:")
        print(f"  {row['response']}")
        print(f"\nParsed: {row['parsed_answer_text']}")
        print(f"Correct: {row['is_correct']} | Refusal: {row['refusal_flag']} | Latency: {row['latency_ms']:.1f}ms")
        
        if row['parsed_answer_json']:
            judgeval = json.loads(row['parsed_answer_json'])
            print(f"Judge Eval: {judgeval}")

## Section 3: Activation Analysis

In [ ]:
# ============================================================================
# Activation Metadata and File Analysis
# ============================================================================

if prerequisites.get('activation_capture', False):
    # Get activation metadata
    activation_meta = pd.read_sql_query("""
        SELECT 
            layer_index,
            component,
            COUNT(*) as count,
            MIN(created_at) as first_capture,
            MAX(created_at) as last_capture
        FROM activation_metadata
        WHERE run_id = ?
        GROUP BY layer_index, component
        ORDER BY layer_index, component
    """, trace_db.conn, params=(RUN_ID,))
    
    print("🔬 Activation Metadata Summary:")
    print(activation_meta.to_string(index=False))
    
    # List activation files
    if ACTIVATIONS_DIR.exists():
        activation_files = sorted(ACTIVATIONS_DIR.glob("*.safetensors"))
        print(f"\n📁 Found {len(activation_files)} activation file(s)")
        
        if activation_files and SAFETENSORS_AVAILABLE:
            # Load a sample file to inspect structure
            sample_file = activation_files[0]
            print(f"\n📂 Sample file: {sample_file.name}")
            
            try:
                tensors = load_safetensors(str(sample_file))
                keys = list(tensors.keys())
                print(f"   Tensor keys: {len(keys)}")
                print(f"   First 10 keys: {keys[:10]}")
                
                # Show tensor shapes
                print(f"\n   Tensor shapes (first 5):")
                for key in keys[:5]:
                    tensor = tensors[key]
                    if isinstance(tensor, torch.Tensor):
                        print(f"     {key}: {tensor.shape} ({tensor.dtype})")
            except Exception as e:
                print(f"   ⚠️  Error loading file: {e}")
    else:
        print("❌ Activations directory not found")
else:
    print("❌ No activation capture data available for this run")

In [ ]:
# ============================================================================
# Load Sample Activations for Specific Trial
# ============================================================================

if prerequisites.get('activation_capture', False) and SAFETENSORS_AVAILABLE:
    # Get a sample trial with activations
    trial_with_activations = pd.read_sql_query("""
        SELECT 
            t.trial_id,
            t.variant,
            s.time_step,
            s.agent_id,
            c.name as condition_name,
            i.question
        FROM conformity_trials t
        JOIN conformity_trial_steps s ON s.trial_id = t.trial_id
        JOIN conformity_conditions c ON c.condition_id = t.condition_id
        JOIN conformity_items i ON i.item_id = t.item_id
        WHERE t.run_id = ?
        LIMIT 1
    """, trace_db.conn, params=(RUN_ID,))
    
    if len(trial_with_activations) > 0:
        sample_trial = trial_with_activations.iloc[0]
        trial_id = sample_trial['trial_id']
        time_step = sample_trial['time_step']
        agent_id = sample_trial['agent_id']
        
        print(f"📊 Sample Trial: {trial_id[:8]}...")
        print(f"   Variant: {sample_trial['variant']}")
        print(f"   Condition: {sample_trial['condition_name']}")
        print(f"   Question: {sample_trial['question']}")
        print(f"   Time Step: {time_step}, Agent: {agent_id}")
        
        # Get activation records for this trial
        activations = pd.read_sql_query("""
            SELECT 
                layer_index,
                component,
                shard_file_path,
                tensor_key,
                shape_json,
                dtype
            FROM activation_metadata
            WHERE run_id = ? AND time_step = ? AND agent_id = ?
            ORDER BY layer_index
        """, trace_db.conn, params=(RUN_ID, time_step, agent_id))
        
        print(f"\n🔬 Found {len(activations)} activation records")
        
        # Load activations for a few layers
        if len(activations) > 0:
            print(f"\n📈 Sample Activation Vectors:")
            for idx, act_row in activations.head(5).iterrows():
                shard_path = Path(RUN_DIR) / act_row['shard_file_path']
                if shard_path.exists():
                    try:
                        tensors = load_safetensors(str(shard_path))
                        vec = tensors[act_row['tensor_key']]
                        if isinstance(vec, torch.Tensor):
                            print(f"   Layer {act_row['layer_index']} ({act_row['component']}):")
                            print(f"     Shape: {vec.shape}")
                            print(f"     Dtype: {vec.dtype}")
                            print(f"     Norm: {vec.float().norm().item():.4f}")
                            print(f"     Mean: {vec.float().mean().item():.4f}")
                            print(f"     Std: {vec.float().std().item():.4f}")
                    except Exception as e:
                        print(f"   ⚠️  Error loading layer {act_row['layer_index']}: {e}")
else:
    print("⏭️  Skipping activation loading (not available or safetensors not installed)")

## Section 4: Probe Analysis

In [ ]:
# ============================================================================
# Probe Information
# ============================================================================

if prerequisites.get('probes', False):
    probes = pd.read_sql_query("""
        SELECT 
            probe_id,
            probe_kind,
            model_id,
            layers_json,
            component,
            token_position,
            artifact_path,
            metrics_json
        FROM conformity_probes
        WHERE run_id = ?
        ORDER BY created_at
    """, trace_db.conn, params=(RUN_ID,))
    
    print("🔍 Trained Probes:")
    for idx, probe in probes.iterrows():
        print(f"\n--- Probe {idx + 1} ---")
        print(f"ID: {probe['probe_id'][:8]}...")
        print(f"Kind: {probe['probe_kind']}")
        print(f"Model: {probe['model_id']}")
        print(f"Component: {probe['component']}")
        print(f"Token Position: {probe['token_position']}")
        print(f"Artifact: {probe['artifact_path']}")
        
        if probe['metrics_json']:
            metrics = json.loads(probe['metrics_json'])
            print(f"Metrics: {json.dumps(metrics, indent=2)}")
        
        # Check if artifact exists
        artifact_path = Path(RUN_DIR) / probe['artifact_path']
        if artifact_path.exists():
            print(f"✅ Artifact file exists")
            if SAFETENSORS_AVAILABLE:
                try:
                    weights = load_safetensors(str(artifact_path))
                    print(f"   Weight keys: {list(weights.keys())[:10]}...")
                    print(f"   Total keys: {len(weights.keys())}")
                except Exception as e:
                    print(f"   ⚠️  Error loading: {e}")
        else:
            print(f"❌ Artifact file not found")
else:
    print("❌ No probes found for this run")

In [ ]:
# ============================================================================
# Probe Projections Analysis
# ============================================================================

if prerequisites.get('probe_projections', False):
    # Get projection statistics
    projections = pd.read_sql_query("""
        SELECT 
            pr.probe_id,
            p.probe_kind,
            pr.layer_index,
            COUNT(*) as projection_count,
            AVG(pr.value_float) as mean_projection,
            AVG(ABS(pr.value_float)) as mean_abs_projection,
            MIN(pr.value_float) as min_projection,
            MAX(pr.value_float) as max_projection
        FROM conformity_probe_projections pr
        JOIN conformity_probes p ON p.probe_id = pr.probe_id
        WHERE p.run_id = ?
        GROUP BY pr.probe_id, pr.layer_index
        ORDER BY pr.probe_id, pr.layer_index
    """, trace_db.conn, params=(RUN_ID,))
    
    print("📊 Probe Projection Statistics:")
    print(projections.to_string(index=False))
    
    # Plot projections by layer for each probe
    if len(projections) > 0:
        probe_kinds = projections['probe_kind'].unique()
        n_probes = len(probe_kinds)
        
        if n_probes > 0:
            fig, axes = plt.subplots(1, min(2, n_probes), figsize=(15, 5))
            if n_probes == 1:
                axes = [axes]
            
            for idx, probe_kind in enumerate(probe_kinds[:2]):
                probe_data = projections[projections['probe_kind'] == probe_kind]
                
                ax = axes[idx] if idx < len(axes) else axes[0]
                ax.plot(probe_data['layer_index'], probe_data['mean_projection'], 
                       marker='o', label=probe_kind, linewidth=2)
                ax.set_xlabel('Layer Index')
                ax.set_ylabel('Mean Projection')
                ax.set_title(f'{probe_kind.capitalize()} Probe Projections by Layer')
                ax.legend()
                ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            # Sample projections for specific trials
            print("\n📋 Sample Projections by Trial:")
            sample_projections = pd.read_sql_query("""
                SELECT 
                    t.trial_id,
                    t.variant,
                    c.name as condition_name,
                    p.probe_kind,
                    pr.layer_index,
                    pr.value_float
                FROM conformity_probe_projections pr
                JOIN conformity_trials t ON t.trial_id = pr.trial_id
                JOIN conformity_probes p ON p.probe_id = pr.probe_id
                JOIN conformity_conditions c ON c.condition_id = t.condition_id
                WHERE p.run_id = ? AND pr.layer_index IN (10, 15, 20)
                ORDER BY t.created_at
                LIMIT 20
            """, trace_db.conn, params=(RUN_ID,))
            
            print(sample_projections.to_string(index=False))
else:
    print("❌ No probe projections found for this run")

## Section 5: Logit Lens Analysis

In [ ]:
# ============================================================================
# Logit Lens Analysis
# ============================================================================

if prerequisites.get('logit_lens', False):
    # Get logit lens statistics
    logit_stats = pd.read_sql_query("""
        SELECT 
            layer_index,
            COUNT(*) as count,
            COUNT(DISTINCT trial_id) as unique_trials
        FROM conformity_logit_lens
        WHERE trial_id IN (SELECT trial_id FROM conformity_trials WHERE run_id = ?)
        GROUP BY layer_index
        ORDER BY layer_index
    """, trace_db.conn, params=(RUN_ID,))
    
    print("🔍 Logit Lens Statistics:")
    print(logit_stats.to_string(index=False))
    
    # Get sample top-k predictions
    sample_logit = pd.read_sql_query("""
        SELECT 
            t.trial_id,
            t.variant,
            c.name as condition_name,
            ll.layer_index,
            ll.token_index,
            ll.topk_json
        FROM conformity_logit_lens ll
        JOIN conformity_trials t ON t.trial_id = ll.trial_id
        JOIN conformity_conditions c ON c.condition_id = t.condition_id
        WHERE t.run_id = ? AND ll.layer_index IN (0, 10, 20, 30)
        ORDER BY t.created_at
        LIMIT 10
    """, trace_db.conn, params=(RUN_ID,))
    
    print("\n📋 Sample Top-K Predictions:")
    for idx, row in sample_logit.iterrows():
        print(f"\n--- Trial {row['trial_id'][:8]}... | Layer {row['layer_index']} ---")
        print(f"Variant: {row['variant']} | Condition: {row['condition_name']}")
        
        topk = json.loads(row['topk_json'])
        print("Top predictions:")
        for i, pred in enumerate(topk[:5]):
            print(f"  {i+1}. '{pred['token']}' (prob: {pred['prob']:.4f})")
else:
    print("❌ No logit lens data found for this run")

## Section 6: Think Token Analysis (for Think Variants)

In [ ]:
# ============================================================================
# Think Token Analysis
# ============================================================================

if prerequisites.get('think_tokens', False):
    # Get think token statistics
    think_stats = pd.read_sql_query("""
        SELECT 
            t.variant,
            COUNT(DISTINCT tt.trial_id) as trials_with_think,
            COUNT(tt.think_id) as total_think_tokens,
            AVG(token_count.count) as avg_tokens_per_trial
        FROM conformity_think_tokens tt
        JOIN conformity_trials t ON t.trial_id = tt.trial_id
        LEFT JOIN (
            SELECT trial_id, COUNT(*) as count
            FROM conformity_think_tokens
            GROUP BY trial_id
        ) token_count ON token_count.trial_id = tt.trial_id
        WHERE t.run_id = ?
        GROUP BY t.variant
    """, trace_db.conn, params=(RUN_ID,))
    
    print("💭 Think Token Statistics:")
    print(think_stats.to_string(index=False))
    
    # Get sample think tokens
    sample_think = pd.read_sql_query("""
        SELECT 
            t.trial_id,
            t.variant,
            c.name as condition_name,
            i.question,
            o.raw_text as response,
            GROUP_CONCAT(tt.token_text, ' ') as think_content
        FROM conformity_think_tokens tt
        JOIN conformity_trials t ON t.trial_id = tt.trial_id
        JOIN conformity_conditions c ON c.condition_id = t.condition_id
        JOIN conformity_items i ON i.item_id = t.item_id
        JOIN conformity_outputs o ON o.trial_id = t.trial_id
        WHERE t.run_id = ? AND t.variant LIKE '%think%'
        GROUP BY t.trial_id
        LIMIT 5
    """, trace_db.conn, params=(RUN_ID,))
    
    print("\n📋 Sample Think Tokens:")
    for idx, row in sample_think.iterrows():
        print(f"\n--- Trial {row['trial_id'][:8]}... ---")
        print(f"Variant: {row['variant']} | Condition: {row['condition_name']}")
        print(f"Question: {row['question']}")
        think_text = row['think_content'] if row['think_content'] else "(no think content)"
        print(f"Think Content: {think_text[:300]}...")
        print(f"Full Response: {row['response'][:300]}...")
else:
    print("⏭️  No think tokens found (this is normal for non-Think variants)")

## Section 7: Intervention Analysis

In [ ]:
# ============================================================================
# Intervention Analysis
# ============================================================================

if prerequisites.get('interventions', False):
    # Get intervention definitions
    interventions = pd.read_sql_query("""
        SELECT 
            intervention_id,
            name,
            alpha,
            target_layers_json,
            component,
            notes
        FROM conformity_interventions
        WHERE run_id = ?
        ORDER BY created_at
    """, trace_db.conn, params=(RUN_ID,))
    
    print("🔧 Intervention Definitions:")
    print(interventions.to_string(index=False))
    
    if prerequisites.get('intervention_results', False):
        # Get intervention results
        intervention_results = pd.read_sql_query("""
            SELECT 
                i.name,
                i.alpha,
                COUNT(*) as total_trials,
                SUM(CASE WHEN r.flipped_to_truth = 1 THEN 1 ELSE 0 END) as flipped_count,
                AVG(CASE WHEN r.flipped_to_truth = 1 THEN 1.0 ELSE 0.0 END) as flip_rate
            FROM conformity_intervention_results r
            JOIN conformity_interventions i ON i.intervention_id = r.intervention_id
            WHERE i.run_id = ?
            GROUP BY i.intervention_id, i.name, i.alpha
            ORDER BY i.alpha
        """, trace_db.conn, params=(RUN_ID,))
        
        print("\n📊 Intervention Results:")
        print(intervention_results.to_string(index=False))
        
        # Sample intervention before/after
        sample_intervention = pd.read_sql_query("""
            SELECT 
                t.trial_id,
                c.name as condition_name,
                i.question,
                i.ground_truth_text,
                o1.raw_text as response_before,
                o1.is_correct as correct_before,
                o2.raw_text as response_after,
                o2.is_correct as correct_after,
                r.flipped_to_truth,
                interv.name as intervention_name,
                interv.alpha
            FROM conformity_intervention_results r
            JOIN conformity_trials t ON t.trial_id = r.trial_id
            JOIN conformity_conditions c ON c.condition_id = t.condition_id
            JOIN conformity_items i ON i.item_id = t.item_id
            JOIN conformity_outputs o1 ON o1.output_id = r.output_id_before
            JOIN conformity_outputs o2 ON o2.output_id = r.output_id_after
            JOIN conformity_interventions interv ON interv.intervention_id = r.intervention_id
            WHERE t.run_id = ? AND r.flipped_to_truth = 1
            LIMIT 5
        """, trace_db.conn, params=(RUN_ID,))
        
        if len(sample_intervention) > 0:
            print("\n📋 Sample Successful Interventions (Flipped to Truth):")
            for idx, row in sample_intervention.iterrows():
                print(f"\n--- Intervention {idx + 1} ---")
                print(f"Intervention: {row['intervention_name']} (alpha={row['alpha']})")
                print(f"Question: {row['question']}")
                print(f"Ground Truth: {row['ground_truth_text']}")
                print(f"Before: {row['response_before']} (correct: {row['correct_before']})")
                print(f"After: {row['response_after']} (correct: {row['correct_after']})")
                print(f"✅ Flipped to truth!")
else:
    print("⏭️  No interventions found for this run")

## Section 8: Behavioral Analysis and Visualizations

In [ ]:
# ============================================================================
# Behavioral Analysis: Accuracy and Conformity Rates
# ============================================================================

# Calculate accuracy by variant and condition
behavioral_analysis = pd.read_sql_query("""
    SELECT 
        t.variant,
        c.name as condition_name,
        COUNT(*) as total_trials,
        SUM(CASE WHEN o.is_correct = 1 THEN 1 ELSE 0 END) as correct_count,
        SUM(CASE WHEN o.is_correct = 0 THEN 1 ELSE 0 END) as incorrect_count,
        AVG(CASE WHEN o.is_correct = 1 THEN 1.0 ELSE 0.0 END) as accuracy,
        AVG(o.latency_ms) as avg_latency_ms
    FROM conformity_trials t
    JOIN conformity_conditions c ON c.condition_id = t.condition_id
    LEFT JOIN conformity_outputs o ON o.trial_id = t.trial_id
    WHERE t.run_id = ? AND o.is_correct IS NOT NULL
    GROUP BY t.variant, c.name
    ORDER BY t.variant, c.name
""", trace_db.conn, params=(RUN_ID,))

print("📊 Behavioral Analysis:")
print(behavioral_analysis.to_string(index=False))

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Accuracy by variant and condition
pivot_acc = behavioral_analysis.pivot(index='variant', columns='condition_name', values='accuracy')
pivot_acc.plot(kind='bar', ax=axes[0, 0], rot=45)
axes[0, 0].set_title('Accuracy by Variant and Condition')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend(title='Condition')
axes[0, 0].grid(True, alpha=0.3)

# 2. Trial counts
pivot_count = behavioral_analysis.pivot(index='variant', columns='condition_name', values='total_trials')
pivot_count.plot(kind='bar', ax=axes[0, 1], rot=45)
axes[0, 1].set_title('Trial Counts by Variant and Condition')
axes[0, 1].set_ylabel('Number of Trials')
axes[0, 1].legend(title='Condition')
axes[0, 1].grid(True, alpha=0.3)

# 3. Latency analysis
pivot_latency = behavioral_analysis.pivot(index='variant', columns='condition_name', values='avg_latency_ms')
pivot_latency.plot(kind='bar', ax=axes[1, 0], rot=45)
axes[1, 0].set_title('Average Latency by Variant and Condition')
axes[1, 0].set_ylabel('Latency (ms)')
axes[1, 0].legend(title='Condition')
axes[1, 0].grid(True, alpha=0.3)

# 4. Conformity rate (accuracy drop from control)
control_data = behavioral_analysis[behavioral_analysis['condition_name'] == 'control']
pressure_data = behavioral_analysis[behavioral_analysis['condition_name'] != 'control']

if len(control_data) > 0 and len(pressure_data) > 0:
    control_acc = control_data.set_index('variant')['accuracy']
    pressure_acc = pressure_data.pivot(index='variant', columns='condition_name', values='accuracy')
    
    conformity_rate = {}
    for variant in control_acc.index:
        if variant in pressure_acc.index:
            for condition in pressure_acc.columns:
                if not pd.isna(pressure_acc.loc[variant, condition]):
                    key = f"{variant}_{condition}"
                    conformity_rate[key] = control_acc[variant] - pressure_acc.loc[variant, condition]
    
    if conformity_rate:
        pd.Series(conformity_rate).plot(kind='bar', ax=axes[1, 1], rot=45)
        axes[1, 1].set_title('Conformity Rate (Control Accuracy - Pressure Accuracy)')
        axes[1, 1].set_ylabel('Accuracy Drop')
        axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Section 9: Export Summary Report

In [ ]:
# ============================================================================
# Generate Summary Report
# ============================================================================

summary = {
    "run_id": RUN_ID,
    "run_dir": str(RUN_DIR),
    "analysis_timestamp": datetime.now().isoformat(),
    "prerequisites": prerequisites,
    "statistics": {
        "total_trials": int(total_trials),
        "total_outputs": int(total_outputs),
        "variants": int(trials_summary['variant'].nunique()),
        "conditions": int(trials_summary['condition_name'].nunique()),
    },
    "behavioral_summary": behavioral_analysis.to_dict('records') if 'behavioral_analysis' in globals() else [],
}

# Save summary
summary_path = Path(RUN_DIR) / "artifacts" / "analysis_summary.json"
summary_path.parent.mkdir(parents=True, exist_ok=True)

with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print(f"✅ Summary report saved to: {summary_path}")

# Display summary
print("\n" + "="*80)
print("📋 ANALYSIS SUMMARY")
print("="*80)
print(f"Run ID: {RUN_ID}")
print(f"Total Trials: {summary['statistics']['total_trials']}")
print(f"Variants: {summary['statistics']['variants']}")
print(f"Conditions: {summary['statistics']['conditions']}")
print(f"\nAvailable Data:")
for name, exists in prerequisites.items():
    status = "✅" if exists else "❌"
    print(f"  {status} {name}")
print("="*80)

In [ ]:
# ============================================================================
# Cleanup
# ============================================================================

trace_db.close()
print("✅ Database connection closed")
print("\n🎉 Analysis complete!")